
Based on https://www.frontiersin.org/journals/immunology/articles/10.3389/fimmu.2021.640725/full

The rationale for the Over Amplification Rate measure
Since out-of-frame TCR/BCR rearrangements do not form a functional receptor, they are not subjected to any specific clonal expansions and selection (Murugan et al., 2012). Being a passenger genomic variation, they change their initial (recombinational) clonal frequencies just randomly following the frequency changes of the second functional (in-frame) TCR/BCR allele present in the same T/B cell clone. According to the TCR/BCR loci rearrangement mechanism, the formation of in-frame and out-of-frame allele combinations in the same cell is also a stochastic and independent process in terms of V- and J-genes frequency. It leads to the conclusion that V- and J-gene frequencies among out-of-frame rearrangements must be sufficiently stable and must be equal to the initial recombination frequencies despite repertoire changes caused by various immune challenges (Figure 1). Thus, reproducible deviation of out-of-frame V- and J-gene frequencies (for the same multiplex PCR primer set) from the initial recombinational frequencies observed in the sequenced repertoire dataset is a result of artificial aberration caused by PCR amplification rather than immune repertoire evolution. Thus out-of-frame clonotypes can be considered a natural calibrator that can be used to measure amplification bias and quantitatively correct immune repertoire data.

# Import

In [ ]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

# Input data

In [ ]:
## multiplex, without UMI

path = '/home/mgikalo/projects/rnrmu_p1/cfse1_reanalyze/mixcr/p1-KRAS-1-mut-CD8-CFSElo-beta-chain.clones_TRB.tsv'

In [ ]:
df1 = pd.read_csv(path, sep='\t')

# transform V- and J-segments format

df1['Vsegm'] = [x.split('*')[0] for x in df1.allVHitsWithScore]
df1['Jsegm'] = [x.split('*')[0] for x in df1.allJHitsWithScore]

# leave only necessary columns in df
df1 = df1[['aaSeqCDR3', 'nSeqCDR3', 'Vsegm', 'Jsegm', 'readCount']]

# manage duplicates
df1 = df1.groupby(['aaSeqCDR3', 'nSeqCDR3', 'Vsegm', 'Jsegm'], as_index=False)\
                .agg({'readCount' : 'sum'})\
                .sort_values('readCount', ascending=False)\
                .reset_index(drop=True)

# remove all clonotypes having 1 read
# because of this:
# "1.8 (for MPlex) and 2.5 (for RACE) reads per out-of-frame clonotype are a minimal sufficient sequencing coverage to get adequate OAR values with an acceptable error rate of ~10%"
df1 = df1.loc[df1.readCount>1]

df1

# Select non-functional clonotypes

In [ ]:
## All clonotypes
len([x for x in df1.aaSeqCDR3])

In [ ]:
## functional clonotypes
len([x for x in df1.aaSeqCDR3 if x.isalpha()])

In [ ]:
## non-functional clonotypes
non_f = [x for x in df1.aaSeqCDR3 if x.isalpha()==0]
len([x for x in df1.aaSeqCDR3 if x.isalpha()==0])

In [ ]:
## how many non-functional clonotypes in each V-segment?
df1.loc[df1.aaSeqCDR3.isin(non_f)]\
        .groupby('Vsegm').agg({'aaSeqCDR3' : 'count'})\
        .sort_values('aaSeqCDR3', ascending=False)

In [ ]:
## how many non-functional clonotypes in each J-segment?
df1.loc[df1.aaSeqCDR3.isin(non_f)]\
        .groupby('Jsegm').agg({'aaSeqCDR3' : 'count'})\
        .sort_values('aaSeqCDR3', ascending=False)

In [ ]:
## df for non-func clonotypes
df_nf = df1.loc[df1.aaSeqCDR3.isin(non_f)].sort_values('readCount').reset_index(drop=True)

## Discard the most abundant non-functional clonotype
df_nf = df_nf.iloc[:-1]
df_nf

In [ ]:
## df for func clonotypes
df_f = df1.loc[~df1.aaSeqCDR3.isin(non_f)].sort_values('readCount').reset_index(drop=True)
df_f

In [ ]:
f = df1[['aaSeqCDR3', 'nSeqCDR3', 'readCount']].loc[~df1.aaSeqCDR3.isin(non_f)].nSeqCDR3.to_list()
f

# Search for in-frame/out of frame pairs with Levenstein distance == 1

Search for in-frame and out-of-frame clone pairs which differ by one indel (Levenshtein distance = 1).
If their ratio is less than 1:500, the smaller clone in pair is discarded, and its count is added to the count of the larger clone
(this procedure guarantees to discard most sequencing errors present in 1 per 1000 nucleotides average)
* предлагаю пренебречь прибавлением каунтов для сокращения вычислений

## Levenstein distance

https://blog.paperspace.com/implementing-levenshtein-distance-word-autocomplete-autocorrect/

In [ ]:
def levenshteinDistanceDP(token1, token2):
    distances = np.zeros((len(token1) + 1, len(token2) + 1))

    for t1 in range(len(token1) + 1):
        distances[t1][0] = t1

    for t2 in range(len(token2) + 1):
        distances[0][t2] = t2
        
    a = 0
    b = 0
    c = 0
    
    for t1 in range(1, len(token1) + 1):
        for t2 in range(1, len(token2) + 1):
            if (token1[t1-1] == token2[t2-1]):
                distances[t1][t2] = distances[t1 - 1][t2 - 1]
            else:
                a = distances[t1][t2 - 1]
                b = distances[t1 - 1][t2]
                c = distances[t1 - 1][t2 - 1]
                
                if (a <= b and a <= c):
                    distances[t1][t2] = a + 1
                elif (b <= a and b <= c):
                    distances[t1][t2] = b + 1
                else:
                    distances[t1][t2] = c + 1

    #printDistances(distances, len(token1), len(token2))
    return distances[len(token1)][len(token2)]

In [ ]:
## split the df into functional and non-functional
## calculate levenstein distances
## if levenstein distance == 1 :
# compare redCounts
# if readCount > 1:500 : delete row in non-func and add it's readCount to the other

In [ ]:
for i, row in df_nf.iterrows():
    for j, rw in df_f.iterrows():
        if levenshteinDistanceDP(row.nSeqCDR3, rw.nSeqCDR3) == 1:
            if row.readCount / rw.readCount > 500:
                df_f = df_f.drop(j)
                row.readCount += rw.readCount
            elif rw.readCount / row.readCount > 500:
                df_nf = df_nf.drop(i)
                rw.readCount += row.readCount

In [ ]:
## assign 0 to func and 1 to non-func clonotypes
df_nf['is_nf'] = 1
df_f['is_nf'] = 0

## join dfs
df = pd.concat([df_f, df_nf]).reset_index(drop=True)
df

# Calculate OARs

## For V-segment

In [ ]:
# calculate OAR for each V-segment
dfv = df.query('is_nf==1')\
        .groupby('Vsegm', as_index=False).agg({'is_nf' : 'sum', 'readCount' : 'sum'})\
        .rename(columns={'is_nf' : 'nonfunc_clones'})
dfv['oar_v'] = (dfv.readCount / dfv.readCount.sum()) / (dfv.nonfunc_clones / dfv.nonfunc_clones.sum())
dfv

## For J-segment (optional)

In [ ]:
# calculate OAR for each J-segment
dfj = df.query('is_nf==1')\
        .groupby('Jsegm', as_index=False).agg({'is_nf' : 'sum', 'readCount' : 'sum'})\
        .rename(columns={'is_nf' : 'nonfunc_clones'})
dfj['oar_j'] = (dfj.readCount / dfj.readCount.sum()) / (dfj.nonfunc_clones / dfj.nonfunc_clones.sum())
dfj

## Total OAR

In [ ]:
# add OARs for V- and J- to main df
df = df.merge(dfv[['Vsegm', 'oar_v']], how='left', on='Vsegm')\
        .merge(dfj[['Jsegm', 'oar_j']], how='left', on='Jsegm')

## if no OAR was calculated for V- or J- replace with 1
df = df.fillna(1)

# calculate the overall OAR for each VJ pair
df['oar'] = df.oar_v * df.oar_j

# calculate adjusted read counts
df['readCount_adj'] = df.readCount / df.oar
df

In [ ]:
by_v = df.groupby('Vsegm', as_index=False).agg({'oar_v' : 'mean'})
by_v

In [ ]:
sns.set(rc={'figure.figsize':(16, 6)})
sns.set_style("whitegrid")
sns.barplot(data=by_v, x='Vsegm', y='oar_v', color='grey')
plt.xticks(rotation=30)
plt.show() 

In [ ]:
by_j = df.groupby('Jsegm', as_index=False).agg({'oar_j' : 'mean'})
by_j

In [ ]:
sns.set(rc={'figure.figsize':(10, 4)})
sns.set_style("whitegrid")
sns.barplot(data=by_j, x='Jsegm', y='oar_j', color='grey')
plt.xticks(rotation=30)
plt.show() 